# Azure AI Agent service - Single agent

<img src="https://learn.microsoft.com/en-us/azure/ai-services/agents/media/agent-service-the-glue.png" width=800>

> https://learn.microsoft.com/en-us/azure/ai-services/agents/

In [1]:
import json
import os
import sys
import time

from azure.ai.agents import AgentsClient
from azure.ai.agents.models import ListSortOrder
from azure.identity import DefaultAzureCredential
from datetime import datetime, timezone, timedelta
from dotenv import load_dotenv
from openai import AzureOpenAI

In [2]:
load_dotenv("azure.env")

True

In [3]:
sys.version

'3.10.14 (main, May  6 2024, 19:42:50) [GCC 11.2.0]'

## Project client

In [4]:
endpoint = os.getenv("PROJECT_ENDPOINT")
credential = DefaultAzureCredential()

project_client = AgentsClient(endpoint=endpoint, credential=credential)

## Single agent

In [5]:
agent = project_client.create_agent(
    model="gpt-4o",
    name="my-agent",
    instructions="You are helpful agent",
)
print(f"Created agent, agent ID: {agent.id}")

Created agent, agent ID: asst_DrTB2OxnfO6MeFrwTqGXrO9S


In [6]:
agent

{'id': 'asst_DrTB2OxnfO6MeFrwTqGXrO9S', 'object': 'assistant', 'created_at': 1749546036, 'name': 'my-agent', 'description': None, 'model': 'gpt-4o', 'instructions': 'You are helpful agent', 'tools': [], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {}, 'metadata': {}, 'response_format': 'auto'}

In [7]:
prompt = "Hello"

In [8]:
# [START create_thread]
thread = project_client.threads.create()
# [END create_thread]
print(f"Created thread, thread ID: {thread.id}")

Created thread, thread ID: thread_uXNRuF4JSjlr7hNB4GpquyhR


In [9]:
# [START create_message]
message = project_client.messages.create(thread_id=thread.id, role="user", content="Hello, tell me a joke")
# [END create_message]
print(f"Created message, message ID: {message.id}")

Created message, message ID: msg_MUUJn3tmJGH9ve6MofB4AoFf


In [10]:
thread

{'id': 'thread_uXNRuF4JSjlr7hNB4GpquyhR', 'object': 'thread', 'created_at': 1749546036, 'metadata': {}, 'tool_resources': {}}

In [11]:
message

{'id': 'msg_MUUJn3tmJGH9ve6MofB4AoFf', 'object': 'thread.message', 'created_at': 1749546037, 'assistant_id': None, 'thread_id': 'thread_uXNRuF4JSjlr7hNB4GpquyhR', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': 'Hello, tell me a joke', 'annotations': []}}], 'attachments': [], 'metadata': {}}

In [12]:
# [START create_run]
run = project_client.runs.create(thread_id=thread.id, agent_id=agent.id)
# [END create_run]
print(f"Created run, run ID: {run.id}")

Created run, run ID: run_vm1gNUXLzhgE2J1cNB6G9flt


In [13]:
# Poll the run as long as run status is queued or in progress
while run.status in ["queued", "in_progress", "requires_action"]:
    # Wait for a second
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"Run status: {run.status}")

    if run.status == "failed":
        print(f"Run error: {run.last_error}")

Run status: completed


In [14]:
run

{'id': 'run_vm1gNUXLzhgE2J1cNB6G9flt', 'object': 'thread.run', 'created_at': 1749546038, 'assistant_id': 'asst_DrTB2OxnfO6MeFrwTqGXrO9S', 'thread_id': 'thread_uXNRuF4JSjlr7hNB4GpquyhR', 'status': 'completed', 'started_at': 1749546039, 'expires_at': None, 'cancelled_at': None, 'failed_at': None, 'completed_at': 1749546039, 'required_action': None, 'last_error': None, 'model': 'gpt-4o', 'instructions': 'You are helpful agent', 'tools': [], 'tool_resources': {}, 'metadata': {}, 'temperature': 1.0, 'top_p': 1.0, 'max_completion_tokens': None, 'max_prompt_tokens': None, 'truncation_strategy': {'type': 'auto', 'last_messages': None}, 'incomplete_details': None, 'usage': {'prompt_tokens': 63, 'completion_tokens': 30, 'total_tokens': 93, 'prompt_token_details': {'cached_tokens': 0}}, 'response_format': 'auto', 'tool_choice': 'auto', 'parallel_tool_calls': True}

In [15]:
run.usage

{'prompt_tokens': 63, 'completion_tokens': 30, 'total_tokens': 93, 'prompt_token_details': {'cached_tokens': 0}}

In [16]:
start = run.started_at
end = run.completed_at
elapsed = end - start

iso_fmt = "%Y-%m-%d %H:%M:%S%z"
print(f"Start   : {start.strftime(iso_fmt)}")
print(f"End     : {end.strftime(iso_fmt)}")
print(f"Elapsed : {elapsed}  " f"({elapsed.total_seconds():.3f} seconds)")

Start   : 2025-06-10 09:00:39+0000
End     : 2025-06-10 09:00:39+0000
Elapsed : 0:00:00  (0.000 seconds)


In [17]:
# [START list_messages]
messages = project_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
for msg in messages:
    if msg.text_messages:
        last_text = msg.text_messages[-1]
        print(f"{msg.role}: {last_text.text.value}")
# [END list_messages]

user: Hello, tell me a joke
assistant: Sure! Here's a classic one for you:

Why don’t skeletons fight each other?

Because they don’t have the guts! 😄


In [18]:
id1 = agent.id

## Another agent

In [19]:
# Create a new agent with different instructions

agent = project_client.create_agent(
    model="gpt-4o-mini",
    name="my-agent",
    instructions="You are a GenAI agent. Always answer in French",
)
print(f"Created agent, agent ID: {agent.id}")

Created agent, agent ID: asst_Soj8PWqT7RSYeEXn9MbPsyXS


In [20]:
# [START create_thread]
thread = project_client.threads.create()
# [END create_thread]
print(f"Created thread, thread ID: {thread.id}")

Created thread, thread ID: thread_cTGpJPLCmzxOasY7BhiqAoDZ


In [21]:
# [START create_message]
message = project_client.messages.create(thread_id=thread.id, role="user", content="What is a GenAI model?")
# [END create_message]
print(f"Created message, message ID: {message.id}")

Created message, message ID: msg_yY3GRICJQB9ycO8t8iS0lJDI


In [22]:
# [START create_run]
run = project_client.runs.create(thread_id=thread.id, agent_id=agent.id)
# [END create_run]
print(f"Created run, run ID: {run.id}")

Created run, run ID: run_QnJr0DppqCU9Fn9XKhHjl404


In [23]:
# Poll the run as long as run status is queued or in progress
while run.status in ["queued", "in_progress", "requires_action"]:
    # Wait for a second
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    # [END create_run]
    print(f"Run status: {run.status}")

    if run.status == "failed":
        print(f"Run error: {run.last_error}")

Run status: in_progress
Run status: completed


In [24]:
run.usage

{'prompt_tokens': 71, 'completion_tokens': 116, 'total_tokens': 187, 'prompt_token_details': {'cached_tokens': 0}}

In [25]:
start = run.started_at
end = run.completed_at
elapsed = end - start

iso_fmt = "%Y-%m-%d %H:%M:%S%z"
print(f"Start   : {start.strftime(iso_fmt)}")
print(f"End     : {end.strftime(iso_fmt)}")
print(f"Elapsed : {elapsed}  " f"({elapsed.total_seconds():.3f} seconds)")

Start   : 2025-06-10 09:00:44+0000
End     : 2025-06-10 09:00:46+0000
Elapsed : 0:00:02  (2.000 seconds)


In [26]:
messages = project_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
for msg in messages:
    if msg.text_messages:
        last_text = msg.text_messages[-1]
        print(f"{msg.role}: {last_text.text.value}")

user: What is a GenAI model?
assistant: Un modèle GenAI, ou Génération d'Intelligence Artificielle, est un type de modèle d'intelligence artificielle conçu pour générer du contenu, comme du texte, des images, de la musique, etc. Ces modèles utilisent des techniques avancées d'apprentissage automatique, telles que les réseaux de neurones, pour créer des réponses ou des créations originales basées sur les données sur lesquelles ils ont été formés. Ils peuvent être utilisés dans divers domaines, notamment la création de contenu, l'assistance virtuelle, la traduction, et bien d'autres.


In [27]:
id2 = agent.id

## List of agents

In [28]:
agents = project_client.list_agents()

for agent in agents:
    print(f"Agent ID: {agent.id}, Name: {agent.name}, Model: {agent.model}, Instructions: {agent.instructions}")

Agent ID: asst_Soj8PWqT7RSYeEXn9MbPsyXS, Name: my-agent, Model: gpt-4o-mini, Instructions: You are a GenAI agent. Always answer in French
Agent ID: asst_DrTB2OxnfO6MeFrwTqGXrO9S, Name: my-agent, Model: gpt-4o, Instructions: You are helpful agent


In [29]:
project_client.delete_agent(id1)
print(f"Deleted agent, agent ID: {id1}")

Deleted agent, agent ID: asst_DrTB2OxnfO6MeFrwTqGXrO9S


In [30]:
project_client.delete_agent(id2)
print(f"Deleted agent, agent ID: {id2}")

Deleted agent, agent ID: asst_Soj8PWqT7RSYeEXn9MbPsyXS
